<a href="https://colab.research.google.com/github/benjaminbrown038/Hugging-Face/blob/main/semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Segmentation in Hugging Face

##### Installs 

In [3]:
%%capture
!pip3 install datasets transformers evaluate

##### Import Libraries

In [5]:
from huggingface_hub import notebook_login
from datasets import load_dataset
import json 
from huggingface_hub import cached_download, hf_hub_url
from transformers import AutoImageProcessor
from torchvision.transforms import ColorJitter
import tensorflow as tf 
import evaluate
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer, create_optimizer, TFAutoModelForSemanticSegmentation, DefaultDataCollator, PushToHubCallback, KerasMetricCallback, pipeline, AutoImageProcessor, TFAutoModelForSemanticSegmentation
import matplotlib.pyplot as plt 
import numpy as np 

##### Import Dataset

In [7]:
ds = load_dataset("scene_parse_150", split = "train[:50]")
ds = ds.train_test_split(test_size = 0.2)
train_ds = ds["train"]
test_ds = ds["test"]
train_ds[0]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/20210 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3352 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset scene_parse_150 downloaded and prepared to /root/.cache/huggingface/datasets/scene_parse_150/scene_parsing/1.0.0/d998c54e1b5c5bad12b4d2ec7e1a5f74eee4c153bc1b089a0001677ae9b3fd75. Subsequent calls will reuse this data.


In [ ]:
repo_id = "huggingface/label-files"
filename = "ade20k-hf-doc-builder.json"

In [ ]:
id2label = json.load(open(cached_download(hf_hub_url(repo_id,filename,repo_type = "dataset")),"r"))
id2label = {int(k): v for k,v in id2label.items()}
label2id = {v: k for k,v in id2label.items()}
num_labels = len(id2label)

In [ ]:
def train_transform():

def val_transforms():

def aug_transforms():

def transforms():

def train_transforms():

def val_transforms():

PyTorch

In [ ]:
def compute_metrics():

Tensorflow

In [ ]:
def compute_metrics():

In [ ]:
model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint,id2label = id2label, label2id = label2id)

training_args = TrainingArguments(
    output_dir
    learning_rate
    num_train
    per_device_train_batch_size
    per_device_eval_batch_size
    save_total_limit
    evaluation_strategy
    save_strategy
    save_steps
    eval_steps
    logging_steps
    eval_accumulation_steps
    remove_unused_columns
    push_to_hub)

trainer = Trainer(
    model = model,
    args = training_args,
    train_datset=train_ds,
    eval_dataset = test_ds,
    compute_metrics = compute_metrics)

trainer.train()
train.push_to_hub()

##### Hyperparameters for Model 

In [ ]:
batch_size = 2 
num_epochs = 50 
num_train_steps = len(train_ds) * num_epochs
learning_rate = 6e-5
weight_decay_rate = .01
optimizer, lr_schedule = create_optimizer(
    init_lr = learning_rate,
    num_train_steps = num_train_steps
    weight_decay_rate = weight_decay_rate
    num_warmup_steps = 0)

In [ ]:
model = TFAutoModelForSemanticSegmentation.from_pretrained(
    checkpoint,
    id2label = id2label
    label2id = label2id)

model.compile(optimizer = optimizer)
data_collator = DefaultDataCollator(return_tensors = "tf")

In [ ]:
tf_train_dataset = train_ds.to_tf_dataset(
    columns = ["pixel_values","label"],
    shuffle = True, 
    batch_size = batch_size, 
    collate_fn = data_collator)

tf_eval_dataset = test_ds.to_tf_dataset(
    columns = ["pixel_values","label"],
    shuffle = True,
    batch_size = batch_size,
    collate_fn = data_collator)

metric_callback = KerasMetricCallback(
    metric_fn = compute_metrics,
    eval_dataset = tf_eval_dataset,
    batch_size = batch_size,
    label_cols = ["labels"])

In [ ]:
push_to_hub_callback = PushToHubCallback(output_dir = "scene_segmenation",tokenizer = image_processor)

callbacks = [metric_callback,push_to_hub_callback]

In [ ]:
model.fit(tf_train_dataset,
          validation_data = tf_eval_dataset,
          callbacks = callbacks,
          epochs = num_epochs)

image = ds[0]["image"]
image

In [ ]:
segmenter = pipeline("image-segmentation",model = "my_awesome_seg_model")
segmenter(image)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoding = image_processor(image,return_tensors="pt")
pixel_values = encoding.pixel_values.to(device)
ouputs = model(pixel_values = pixel_values)
logits = outputs.logits.cpu()
unsampled_logits = nn.functional.interpolate(logits,size = image.size[::-1],
                                             mode = "bilinear",
                                             align_corners = False)

NameError: ignored

In [ ]:
pred_seg = upsamples_logits.argmax(dim=1)[0]

image_processor = AutoImageProcessor.from_pretrained("MariaK/scene_segmentation")
inputs = image_processor(image,return_tensors = "tf")
model = TFAutoModelForSemanticSegmentation.from_pretrained("MariaK/scene_segmentation")
logits = model(**inputs).logits

In [ ]:
logits = tf.transpose(logits,[0,2,3,1])
upsampled_logits = tf.image.resize(
    logits,
    image.size[::-1])

In [ ]:
pred_seg = tf.math.argmax(upsampled_logits,axis = -1,axis = -1)[0]


color_seg = np.zeros(([pred_seg.shape[0],pred_seg.shape[1],3,dtype=np.uint8]))
palette = np.array(ade_palette())
for label, color in enumerate(pallete):
    color_seg[pred_seg == label,:] = color

In [ ]:
color_seg = color_seg[]

img = np.array(image) * 0.5 + color_seg * 0.5
img = img.astype(np.uint8)
plt.figure(figsize = (15,10))
plt.imshow(img)
plt.show()